In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

from omegaconf import OmegaConf

import anndata as ad
import pandas as pd
import numpy as np

repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

## MARS metadata data

In [3]:
mars_plates = pd.read_csv("/home/labs/amit/shuangyi/Project_MM_2023/metadata_MM_MARS_20230713.txt", sep="\t")
mars_plates["Method"] = "MARS"
print(mars_plates.shape)
mars_plates.head()

(1037, 17)


,Amp.Batch.ID,Seq.Batch.ID,Batch.Set.ID,Member,Description,Included,Tissue,Gating,Total.PC,Total.NonPC,Initial.Code,Disease,Project,Cohort,Weizmann.Code,Hospital.Code,Method
0,AB2136,SB113,human_BM,Guy,hip5_PC_1,-,BM,PC,350.0,2.0,hip5,Healthy,Pilot,Healthy,hip1,hip1,MARS
1,AB2137,SB113,human_BM,Guy,hip5_PC_2,-,BM,PC,349.0,NaN,hip5,Healthy,Pilot,Healthy,hip1,hip1,MARS
2,AB2138,SB113,human_BM,Guy,hip5_PC_3,-,BM,PC,352.0,1.0,hip5,Healthy,Pilot,Healthy,hip1,hip1,MARS
3,AB2139,SB113,human_BM,Guy,hip5_hCD45_1,No,BM,CD45,1.0,289.0,hip5,Healthy,Pilot,Healthy,hip1,hip1,MARS
4,AB2140,SB113,human_BM,Guy,hip5_hCD45_2,No,BM,CD45,NaN,308.0,hip5,Healthy,Pilot,Healthy,hip1,hip1,MARS


## SPID metadata data

In [4]:
spid_plates = pd.read_csv("/home/labs/amit/shuangyi/Project_MM_2023/metadata_MM_SPID_20230621.txt", sep="\t")
spid_plates["Method"] = "SPID"
print(spid_plates.shape)
spid_plates.head()

(508, 14)


,Amp.Batch.ID,Seq.Batch.ID,Batch.Set.ID,Included,Tissue,Gating,Total.PC,Total.NonPC,Initial.Code,Disease,Project,Cohort,Weizmann.Code,Method
0,AB5597,SB173,Biomek SPID CIA06 1X 15min CD45_1,-,BM,CD45,1.0,65.0,TLV53,RRMM,TLV,TLV,TLV53,SPID
1,AB5598,SB173,Biomek SPID CIA06 1X 15min All cells,-,BM,NaN,2.0,30.0,TLV53,RRMM,TLV,TLV,TLV53,SPID
2,AB5599,SB173,Biomek SPID CIA06 Mechanic PC1,-,BM,PC,160.0,14.0,TLV53,RRMM,TLV,TLV,TLV53,SPID
3,AB5600,SB173,Biomek SPID CIA06 1X 15min PC_1,-,BM,PC,141.0,56.0,TLV53,RRMM,TLV,TLV,TLV53,SPID
4,AB5601,SB173,Biomek SPID CIA06 1X 15min PC_2,-,BM,PC,146.0,69.0,TLV53,RRMM,TLV,TLV,TLV53,SPID


## combine plates meta data

In [6]:
print(set(mars_plates.columns).difference(set(spid_plates.columns)))
print(set(spid_plates.columns).difference(set(mars_plates.columns)))

{'Hospital.Code', 'Member', 'Description'}
set()


In [7]:
mars_plates = mars_plates[spid_plates.columns]
mars_plates.shape

(1037, 14)

In [8]:
first_columns = ['Amp.Batch.ID', 'Seq.Batch.ID', 'Batch.Set.ID']
other_columns = [col for col in mars_plates.columns if col not in first_columns]
combined_plates = pd.concat([mars_plates, spid_plates])[first_columns + other_columns]
print(combined_plates.shape)
combined_plates.head()

(1545, 14)


,Amp.Batch.ID,Seq.Batch.ID,Batch.Set.ID,Included,Tissue,Gating,Total.PC,Total.NonPC,Initial.Code,Disease,Project,Cohort,Weizmann.Code,Method
0,AB2136,SB113,human_BM,-,BM,PC,350.0,2.0,hip5,Healthy,Pilot,Healthy,hip1,MARS
1,AB2137,SB113,human_BM,-,BM,PC,349.0,NaN,hip5,Healthy,Pilot,Healthy,hip1,MARS
2,AB2138,SB113,human_BM,-,BM,PC,352.0,1.0,hip5,Healthy,Pilot,Healthy,hip1,MARS
3,AB2139,SB113,human_BM,No,BM,CD45,1.0,289.0,hip5,Healthy,Pilot,Healthy,hip1,MARS
4,AB2140,SB113,human_BM,No,BM,CD45,NaN,308.0,hip5,Healthy,Pilot,Healthy,hip1,MARS


In [9]:
combined_plates_path = "/home/labs/amit/noamsh/data/mm_2023/2024_2_13_MARS_SPID_COMBINED.tsv"
combined_plates.to_csv(combined_plates_path, index=False, sep='\t')

mars_plates.to_csv("/home/labs/amit/noamsh/data/mm_2023/2024_2_13_MARS.tsv", index=False, sep='\t')
spid_plates.to_csv("/home/labs/amit/noamsh/data/mm_2023/2024_2_13_SPID.tsv", index=False, sep='\t')

## test - load conbined

In [12]:
config_path = Path(repo_dir, "config.yaml")
conf = OmegaConf.load(config_path)

In [13]:
conf.data_loading.plates.plates_data_path = combined_plates_path
conf.data_loading.plates.plate_id_column_name = "Amp.Batch.ID"
conf.data_loading.sc_sequencing.sc_sequencing_data_dir = "/home/labs/amit/noamsh/data/mm_2023/all_scdata"

In [14]:
from load_sc_data_to_anndata import load_sc_data_to_anndata
adata_control = load_sc_data_to_anndata(conf)

/home/labs/amit/noamsh/repos/MM_2023/data_loading/anndata_loaders.py:62: TqdmWarning: Iterable length 1545 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  adatas = process_map(partial(self._get_single_plate, col_names=col_names),


loading relevant plates:   0%|          | 0/1545 [00:00<?, ?plate/s]

/home/labs/amit/noamsh/.conda/envs/CLL_2024/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [15]:
adata_control

AnnData object with n_obs × n_vars = 593280 × 57874
    obs: 'Amp.Batch.ID', 'Seq.Batch.ID', 'Batch.Set.ID', 'Member', 'Description', 'Included', 'Tissue', 'Gating', 'Total.PC', 'Total.NonPC', 'Initial.Code', 'Disease', 'Project', 'Cohort', 'Weizmann.Code', 'Hospital.Code'